<a href="https://colab.research.google.com/github/camilogomez5467/trabajo-python-Quipux/blob/main/trabajo_python_Quipux.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
from typing import Tuple, Dict
import warnings

warnings.filterwarnings('ignore')
sns.set_style('whitegrid')
plt.rcParams['figure.figsize'] = (12, 6)

URL_DATASET = 'https://archive.ics.uci.edu/ml/machine-learning-databases/heart-disease/processed.cleveland.data'

COLUMN_NAMES = [
    'age', 'sex', 'cp', 'trestbps', 'chol', 'fbs',
    'restecg', 'thalach', 'exang', 'oldpeak', 'slope',
    'ca', 'thal', 'target'
]

df = pd.read_csv(URL_DATASET, names=COLUMN_NAMES, na_values='?')

print("Dataset cargado exitosamente")
print(f"Dimensiones: {df.shape}")
print("\nPrimeras 20 filas del dataset:")
display(df.head(20))

In [ ]:
def explorar_dataset(dataframe: pd.DataFrame) -> None:
    """
    Explora las características principales del dataset.

    Args:
        dataframe: DataFrame de pandas a explorar
    """
    print("=" * 60)
    print("EXPLORACIÓN INICIAL DEL DATASET")
    print("=" * 60)

    print(f"\nInformación general:")
    print(dataframe.info())

    print(f"\nEstadísticas descriptivas:")
    display(dataframe.describe())

    print(f"\nValores nulos por columna:")
    print(dataframe.isnull().sum())

    print(f"\nValores únicos por columna:")
    for col in dataframe.columns:
        print(f"{col}: {dataframe[col].nunique()} valores únicos")

    print(f"\nPrimeras 20 filas:")
    display(dataframe.head(20))


def limpiar_datos(dataframe: pd.DataFrame) -> pd.DataFrame:
    """
    Limpia el dataset eliminando valores nulos y corrigiendo datos.

    Args:
        dataframe: DataFrame original

    Returns:
        DataFrame limpio
    """
    df_limpio = dataframe.copy()

    print("\n" + "=" * 60)
    print("LIMPIEZA DE DATOS")
    print("=" * 60)

    valores_nulos = df_limpio.isnull().sum()
    print(f"\nValores nulos encontrados:")
    print(valores_nulos[valores_nulos > 0])

    print("\nEXPLICACIÓN: Eliminamos filas con valores nulos en 'ca' y 'thal'")
    print("porque representan menos del 2% del dataset y son variables")
    print("clínicas importantes que no pueden ser imputadas sin riesgo.")

    filas_antes = len(df_limpio)
    df_limpio = df_limpio.dropna()
    filas_despues = len(df_limpio)

    print(f"\nFilas eliminadas: {filas_antes - filas_despues}")
    print(f"Porcentaje de datos eliminados: {((filas_antes - filas_despues) / filas_antes) * 100:.2f}%")

    print("\nEXPLICACIÓN: La variable target se convierte de multinivel (0-4)")
    print("a binaria (0-1) para simplificar el análisis enfocándose")
    print("en presencia/ausencia de enfermedad.")

    df_limpio['target'] = (df_limpio['target'] > 0).astype(int)

    print("\nVariable 'target' convertida a binaria:")
    print(df_limpio['target'].value_counts())

    print("\nVerificación de rangos de valores:")

    valores_atipicos_edad = df_limpio[(df_limpio['age'] < 0) | (df_limpio['age'] > 120)]
    print(f"Valores atípicos en edad: {len(valores_atipicos_edad)}")

    valores_cero_colesterol = df_limpio[df_limpio['chol'] == 0]
    print(f"Valores de colesterol en 0: {len(valores_cero_colesterol)}")

    if len(valores_cero_colesterol) > 0:
        print("\nEXPLICACIÓN: Eliminamos filas con colesterol = 0 porque")
        print("es biológicamente imposible y representa error de medición.")
        df_limpio = df_limpio[df_limpio['chol'] > 0]
        print(f"Filas con colesterol = 0 eliminadas: {len(valores_cero_colesterol)}")

    valores_atipicos_presion = df_limpio[(df_limpio['trestbps'] < 80) | (df_limpio['trestbps'] > 200)]
    print(f"Valores atípicos en presión arterial: {len(valores_atipicos_presion)}")

    print("\nDataset limpio - Primeras 20 filas:")
    display(df_limpio.head(20))

    return df_limpio


explorar_dataset(df)
df_clean = limpiar_datos(df)


In [ ]:
def aplicar_transformaciones(dataframe: pd.DataFrame) -> Dict[str, pd.DataFrame]:
    """
    Aplica transformaciones y filtros al dataset.

    Args:
        dataframe: DataFrame limpio

    Returns:
        Diccionario con diferentes subconjuntos de datos
    """
    transformaciones = {}

    print("" + "=" * 60)
    print("TRANSFORMACIONES Y FILTRACIÓN DE DATOS")
    print("=" * 60)

    dataframe['age_group'] = pd.cut(
        dataframe['age'],
        bins=[0, 40, 50, 60, 100],
        labels=['<40', '40-50', '50-60', '>60']
    )

    dataframe['chol_category'] = pd.cut(
        dataframe['chol'],
        bins=[0, 200, 240, 500],
        labels=['Normal', 'Límite Alto', 'Alto']
    )

    print("\nDataset con nuevas variables - Primeras 20 filas:")
    display(dataframe.head(20))

    transformaciones['mayores_50'] = dataframe[dataframe['age'] > 50]
    print(f" FILTRO 1 - Pacientes mayores de 50 años: {len(transformaciones['mayores_50'])}")
    display(transformaciones['mayores_50'].head(20))

    transformaciones['mujeres_hipertension'] = dataframe[
        (dataframe['sex'] == 0) & (dataframe['trestbps'] > 140)
    ]
    print(f" FILTRO 2 - Mujeres con hipertensión: {len(transformaciones['mujeres_hipertension'])}")
    display(transformaciones['mujeres_hipertension'].head(20))

    transformaciones['hombres_colesterol_alto'] = dataframe[
        (dataframe['sex'] == 1) & (dataframe['chol'] > 240)
    ]
    print(f" FILTRO 3 - Hombres con colesterol alto: {len(transformaciones['hombres_colesterol_alto'])}")
    display(transformaciones['hombres_colesterol_alto'].head(20))

    transformaciones['con_enfermedad'] = dataframe[dataframe['target'] == 1]
    print(f" FILTRO 4 - Pacientes con enfermedad cardíaca: {len(transformaciones['con_enfermedad'])}")
    display(transformaciones['con_enfermedad'].head(20))

    return transformaciones


df_clean_transformed = df_clean.copy()
subgrupos = aplicar_transformaciones(df_clean_transformed)


In [ ]:
def calcular_estadisticas_agregadas(dataframe: pd.DataFrame) -> pd.DataFrame:
    """
    Calcula estadísticas agregadas por diferentes grupos.

    Args:
        dataframe: DataFrame con datos transformados

    Returns:
        DataFrame consolidado con estadísticas
    """
    print("" + "=" * 60)
    print("ESTADÍSTICAS AGREGADAS Y AGRUPACIONES")
    print("=" * 60)

    print("\n--- Estadísticas por Sexo ---")
    stats_sex = dataframe.groupby('sex').agg({
        'age': ['mean', 'std'],
        'chol': ['mean', 'std'],
        'trestbps': ['mean', 'std'],
        'thalach': ['mean', 'std'],
        'target': ['sum', 'mean']
    }).round(2)
    stats_sex.columns = ['_'.join(col) for col in stats_sex.columns]
    display(stats_sex)

    print("\n--- Estadísticas por Grupo de Edad ---")
    stats_age = dataframe.groupby('age_group').agg({
        'chol': 'mean',
        'trestbps': 'mean',
        'thalach': 'mean',
        'target': ['sum', 'mean']
    }).round(2)
    stats_age.columns = ['_'.join(col) for col in stats_age.columns]
    display(stats_age)

    print("\n--- Distribución de Enfermedad por Sexo ---")
    stats_sex_disease = pd.crosstab(
        dataframe['sex'],
        dataframe['target'],
        normalize='index'
    ).round(3) * 100
    stats_sex_disease.columns = ['Sin Enfermedad (%)', 'Con Enfermedad (%)']
    stats_sex_disease.index = ['Mujer', 'Hombre']
    display(stats_sex_disease)

    consolidado = dataframe.groupby(['sex', 'age_group']).agg({
        'age': 'mean',
        'chol': 'mean',
        'trestbps': 'mean',
        'thalach': 'mean',
        'target': ['sum', 'count', 'mean']
    }).round(2)

    consolidado.columns = ['_'.join(col) for col in consolidado.columns]
    consolidado = consolidado.rename(columns={
        'target_sum': 'total_con_enfermedad',
        'target_count': 'total_pacientes',
        'target_mean': 'tasa_enfermedad'
    })

    return consolidado


tabla_consolidada = calcular_estadisticas_agregadas(df_clean_transformed)
print("\n--- TABLA CONSOLIDADA FINAL ---")
display(tabla_consolidada)


In [ ]:
def crear_visualizaciones(dataframe: pd.DataFrame) -> None:
    """
    Crea gráficas para análisis visual del dataset.

    Args:
        dataframe: DataFrame con datos a visualizar
    """
    print("" + "=" * 60)
    print("VISUALIZACIÓN DE DATOS")
    print("=" * 60)

    plt.figure(figsize=(12, 5))

    plt.subplot(1, 2, 1)
    plt.hist(dataframe['age'], bins=20, color='skyblue', edgecolor='black')
    plt.title('Distribución de Edad de los Pacientes', fontsize=14, fontweight='bold')
    plt.xlabel('Edad (años)')
    plt.ylabel('Frecuencia')
    plt.grid(axis='y', alpha=0.3)

    plt.subplot(1, 2, 2)
    dataframe.boxplot(column='age', by='target', figsize=(6, 5))
    plt.title('Distribución de Edad por Presencia de Enfermedad')
    plt.suptitle('')
    plt.xlabel('Enfermedad Cardíaca (0=No, 1=Sí)')
    plt.ylabel('Edad (años)')

    plt.tight_layout()
    plt.show()

    print(" Interpretación - Distribución de Edad:")
    print("- La edad promedio de los pacientes es alrededor de 54 años")
    print("- La mayoría de pacientes tienen entre 45 y 65 años")
    print("- Los pacientes con enfermedad cardíaca tienden a ser ligeramente mayores")

    plt.figure(figsize=(14, 5))

    plt.subplot(1, 2, 1)
    for target_val in [0, 1]:
        data = dataframe[dataframe['target'] == target_val]['chol']
        plt.hist(data, bins=20, alpha=0.6,
                label=f'{"Con" if target_val == 1 else "Sin"} Enfermedad')
    plt.title('Distribución de Colesterol por Enfermedad Cardíaca',
             fontsize=14, fontweight='bold')
    plt.xlabel('Colesterol (mg/dl)')
    plt.ylabel('Frecuencia')
    plt.legend()
    plt.grid(axis='y', alpha=0.3)

    plt.subplot(1, 2, 2)
    dataframe.boxplot(column='chol', by='target')
    plt.title('Niveles de Colesterol vs Enfermedad Cardíaca')
    plt.suptitle('')
    plt.xlabel('Enfermedad Cardíaca (0=No, 1=Sí)')
    plt.ylabel('Colesterol (mg/dl)')

    plt.tight_layout()
    plt.show()

    print("Interpretación - Colesterol:")
    print("- El colesterol promedio es similar entre grupos con y sin enfermedad")
    print("- Colesterol alto (>240 mg/dl) es un factor de riesgo pero no determinante")
    print("- Se observan valores atípicos en ambos grupos")

    plt.figure(figsize=(14, 5))

    plt.subplot(1, 2, 1)
    sex_disease = pd.crosstab(dataframe['sex'], dataframe['target'])
    sex_disease.index = ['Mujer', 'Hombre']
    sex_disease.columns = ['Sin Enfermedad', 'Con Enfermedad']
    sex_disease.plot(kind='bar', stacked=False, color=['#66c2a5', '#fc8d62'])
    plt.title('Presencia de Enfermedad Cardíaca por Sexo',
             fontsize=14, fontweight='bold')
    plt.xlabel('Sexo')
    plt.ylabel('Número de Pacientes')
    plt.xticks(rotation=0)
    plt.legend(title='Estado')
    plt.grid(axis='y', alpha=0.3)

    plt.subplot(1, 2, 2)
    proportions = pd.crosstab(dataframe['sex'], dataframe['target'], normalize='index')
    proportions.index = ['Mujer', 'Hombre']
    proportions.columns = ['Sin Enfermedad', 'Con Enfermedad']
    proportions.plot(kind='bar', stacked=True, color=['#66c2a5', '#fc8d62'])
    plt.title('Proporción de Enfermedad Cardíaca por Sexo (%)',
             fontsize=14, fontweight='bold')
    plt.xlabel('Sexo')
    plt.ylabel('Proporción')
    plt.xticks(rotation=0)
    plt.legend(title='Estado')
    plt.grid(axis='y', alpha=0.3)

    plt.tight_layout()
    plt.show()

    print("Interpretación - Diferencias por Sexo:")
    print("- Los hombres tienen mayor prevalencia de enfermedad cardíaca")
    print("- El dataset tiene más pacientes masculinos")
    print("- Las mujeres muestran tasas más bajas de enfermedad en este estudio")

    plt.figure(figsize=(12, 8))

    numeric_cols = dataframe.select_dtypes(include=[np.number]).columns
    correlation_matrix = dataframe[numeric_cols].corr()

    sns.heatmap(correlation_matrix, annot=True, fmt='.2f', cmap='coolwarm',
                center=0, square=True, linewidths=1)
    plt.title('Matriz de Correlación de Variables Clínicas',
             fontsize=14, fontweight='bold')
    plt.tight_layout()
    plt.show()

    print("Interpretación - Correlaciones:")
    print("- Correlación negativa entre edad y frecuencia cardíaca máxima")
    print("- El tipo de dolor torácico (cp) tiene correlación con enfermedad")
    print("- Múltiples factores combinados determinan el riesgo")


crear_visualizaciones(df_clean_transformed)


In [ ]:
def generar_analisis_final(dataframe: pd.DataFrame,
                          consolidado: pd.DataFrame) -> None:
    """
    Genera el análisis final y conclusiones del estudio.

    Args:
        dataframe: DataFrame completo
        consolidado: Tabla consolidada de estadísticas
    """
    print("" + "=" * 60)
    print("ANÁLISIS FINAL Y CONCLUSIONES")
    print("=" * 60)

    total_pacientes = len(dataframe)
    con_enfermedad = dataframe['target'].sum()
    tasa_enfermedad = (con_enfermedad / total_pacientes) * 100

    edad_promedio_enfermo = dataframe[dataframe['target'] == 1]['age'].mean()
    edad_promedio_sano = dataframe[dataframe['target'] == 0]['age'].mean()

    print("ESTADÍSTICAS CLAVE:")
    print(f"- Total de pacientes analizados: {total_pacientes}")
    print(f"- Pacientes con enfermedad cardíaca: {con_enfermedad} ({tasa_enfermedad:.1f}%)")
    print(f"- Edad promedio (con enfermedad): {edad_promedio_enfermo:.1f} años")
    print(f"- Edad promedio (sin enfermedad): {edad_promedio_sano:.1f} años")

    print(" HALLAZGOS PRINCIPALES:")
    print("FACTORES DE RIESGO IDENTIFICADOS:")
    print("   - Edad avanzada (>55 años) aumenta significativamente el riesgo")
    print("   - El sexo masculino presenta mayor prevalencia de enfermedad")
    print("   - El tipo de dolor torácico es un indicador importante")
    print("   - La presencia de angina inducida por ejercicio es significativa")

    print(" PATRONES DEMOGRÁFICOS:")
    print("   - Mayor concentración de casos entre 50-65 años")
    print("   - Hombres representan ~2/3 de los casos con enfermedad")
    print("   - La hipertensión es un factor común pero no determinante único")

    print("VARIABLES CLÍNICAS RELEVANTES:")
    print("   - Frecuencia cardíaca máxima: correlación negativa con enfermedad")
    print("   - Colesterol: factor de riesgo cuando está elevado (>240 mg/dl)")
    print("   - Depresión ST (oldpeak): indicador importante de isquemia")


    print("CONCLUSIONES Y RECOMENDACIONES:")
    print("✓ PARA PREVENCIÓN:")
    print("  - Implementar programas de screening para mayores de 50 años")
    print("  - Enfoque especial en población masculina de alto riesgo")
    print("  - Control regular de colesterol y presión arterial")
    print("  - Promover actividad física controlada y monitoreo cardíaco")

    print("✓ PARA POLÍTICAS DE SALUD:")
    print("  - Campañas educativas sobre factores de riesgo modificables")
    print("  - Programas de detección temprana en atención primaria")
    print("  - Facilitar acceso a exámenes cardíacos básicos")
    print("  - Crear protocolos de estratificación de riesgo")

    print("✓ PARA INVESTIGACIÓN FUTURA:")
    print("  - Estudiar interacción entre múltiples factores de riesgo")
    print("  - Analizar diferencias de género más detalladamente")
    print("  - Incluir variables socioeconómicas y de estilo de vida")
    print("  - Desarrollar modelos predictivos con machine learning")

    print("LIMITACIONES DEL ESTUDIO:")
    print("  - Dataset relativamente pequeño (~300 pacientes)")
    print("  - Datos de una sola ubicación geográfica (Cleveland)")
    print("  - Algunas variables clínicas no incluidas")
    print("  - Necesidad de validación con datos más recientes")


generar_analisis_final(df_clean_transformed, tabla_consolidada)


In [ ]:
tabla_consolidada.to_csv('tabla_consolidada_heart_disease.csv')
print("Tabla consolidada guardada como 'tabla_consolidada_heart_disease.csv'")
print(f"Dimensiones de la tabla consolidada: {tabla_consolidada.shape}")
print("Vista previa completa de la tabla consolidada:")
display(tabla_consolidada)
